In [4]:
DATA_URL = "https://s3.wasabisys.com/iguazio/data/market-palce/arc_to_parquet/higgs-sample.csv.gz"
# original large file

In [14]:
from mlrun import import_function, set_environment
import os

set_environment( artifact_path='./artifacts')


('default', '/home/jovyan/aviaIguazio/functions/arc_to_parquet/artifacts')

In [18]:
from mlrun import import_function

fn = import_function("hub://arc_to_parquet")
fn.run(params={"key": "higgs-sample"},
           handler="arc_to_parquet",
           inputs={"archive_url": DATA_URL},          
           ).show()



> 2021-07-25 12:43:35,948 [info] starting run arc-to-parquet-arc_to_parquet uid=c1e93f3037b04a139a390fb1b455c09a DB=http://mlrun-api:8080
> 2021-07-25 12:43:36,037 [info] Job is running in the background, pod: arc-to-parquet-arc-to-parquet-nhldg
> 2021-07-25 12:43:39,274 [info] downloading https://s3.wasabisys.com/iguazio/data/market-palce/arc_to_parquet/higgs-sample.csv.gz to local tmp
> 2021-07-25 12:43:39,434 [info] destination file does not exist, downloading
> 2021-07-25 12:43:39,562 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...b455c09a,0,Jul 25 12:43:39,completed,arc-to-parquet-arc_to_parquet,kind=jobowner=jovyanhost=arc-to-parquet-arc-to-parquet-nhldg,archive_url,key=higgs-sample,,higgs-sample


to track results use .show() or .logs() or in CLI: 
!mlrun get run c1e93f3037b04a139a390fb1b455c09a --project default , !mlrun logs c1e93f3037b04a139a390fb1b455c09a --project default
> 2021-07-25 12:43:42,147 [info] run executed, status=completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...b455c09a,0,Jul 25 12:43:39,completed,arc-to-parquet-arc_to_parquet,kind=jobowner=jovyanhost=arc-to-parquet-arc-to-parquet-nhldg,archive_url,key=higgs-sample,,higgs-sample


In [21]:
import pandas as pd
pd.read_parquet('artifacts/higgs-sample.parquet')


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/higgs-sample.parquet'